In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

data = pd.read_csv(r"./Tweets.csv")
data.head(5)

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [68]:
X_train, X_test, y_train, y_test = train_test_split(data["text"], data["sentiment"], test_size=0.2, stratify=data["sentiment"])

len(X_train), len(X_test)

(21984, 5497)

In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    if not isinstance(text, str) or text.strip() == "":
        return ""
    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w.lower() not in stop_words]
    tokens = [lemmatizer.lemmatize(w.lower()) for w in tokens]
    tokens = " ".join(tokens)
    return tokens

X_train = X_train.fillna("").apply(preprocess)
X_test = X_test.fillna("").apply(preprocess)
print(X_train[:5])

vectorizer = TfidfVectorizer(sublinear_tf = True,
                             use_idf = True,
                             ngram_range=(1, 2))

X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)

22351                                 shot still alive ...
6629     cant wait & & cant wait see 3d concert experie...
8236          ` make . manager switched schedule ` closing
12354                                      yes pm pm right
16287    thats , rub ur seeing morrissey ! il go cry co...
Name: text, dtype: object


In [70]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

X_train_vectors = torch.tensor(X_train_vectors.toarray(), dtype=torch.float32)
X_test_vectors = torch.tensor(X_test_vectors.toarray(), dtype=torch.float32)

y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train_vectors, y_train)
test_dataset = TensorDataset(X_test_vectors, y_test)

X_train_vectors.shape

torch.Size([21984, 128155])

In [74]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

class Network(nn.Module):
    def __init__(self, first_dim):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(first_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 3),
        )
    
    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

model = Network(X_train_vectors.shape[1]).to(device)

print(model)

Using mps device
Network(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=128155, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=512, out_features=128, bias=True)
    (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.5, inplace=False)
    (8): Linear(in_features=128, out_features=3, bias=True)
  )
)


In [ ]:
N = 100
train_dataloader = DataLoader(train_dataset, batch_size=N)
test_dataloader = DataLoader(test_dataset, batch_size=N)

def train(model, dataloader, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for i, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i % 100 == 0:
            loss, current = loss.item(), i * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(model, dataloader, loss_fn):
    size = len(dataloader.dataset)
    num_N = len(dataloader)
    
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_N
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [76]:
epochs = 5

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-3)

for i in range(epochs):
    print(f"Epoch {i+1}\n-------------------------------")
    train(model, train_dataloader, loss_fn, optimizer)
    test(model, test_dataloader, loss_fn)

print("Done!")

Epoch 1
-------------------------------
loss: 1.199392  [    0/21984]
loss: 0.925080  [10000/21984]
loss: 1.039364  [20000/21984]
Test Error: 
 Accuracy: 66.6%, Avg loss: 0.013548 

Epoch 2
-------------------------------
loss: 0.645393  [    0/21984]
loss: 0.422892  [10000/21984]
loss: 0.441784  [20000/21984]
Test Error: 
 Accuracy: 67.2%, Avg loss: 0.013248 

Epoch 3
-------------------------------
loss: 0.272650  [    0/21984]
loss: 0.186068  [10000/21984]
loss: 0.181614  [20000/21984]
Test Error: 
 Accuracy: 66.5%, Avg loss: 0.014094 

Epoch 4
-------------------------------
loss: 0.119349  [    0/21984]
loss: 0.108141  [10000/21984]
loss: 0.073859  [20000/21984]
Test Error: 
 Accuracy: 65.7%, Avg loss: 0.013912 

Epoch 5
-------------------------------
loss: 0.082445  [    0/21984]
loss: 0.056631  [10000/21984]
loss: 0.091096  [20000/21984]
Test Error: 
 Accuracy: 65.6%, Avg loss: 0.014548 

Done!


In [97]:
text = input("Text: ")
text = preprocess(text)
text = vectorizer.transform([text])
text = torch.tensor(text.toarray(), dtype=torch.float32).to(device)

pred = model(text)
pred = le.inverse_transform([pred.argmax(1).item()])
pred[0]

'neutral'